In [0]:
# Adrian's ffmpeg approach with an offset pertaining to the known time within the 30 min video when a fish occurs.

import subprocess

def extract_clip_ffmpeg(input_path: str, offset_seconds: int, output_path: str, duration: int = 10, quality: int = 4):
    """
    Extracts a video clip using ffmpeg from the input_path starting at offset_seconds,
    for the specified duration, and saves it to output_path.
    """
    command = [
        "ffmpeg",
        "-n",                 # Do not overwrite
        "-v", "quiet",        # Keep console output to minimum 
        "-stats",             # Show encoding stats
        "-i", input_path,     # Input file location
        "-ss", str(offset_seconds),  # Start time of event
        "-t", str(duration),         # Duration
        "-q", str(quality),          # Quality level
        output_path           # Output file location
    ]

    try:
        subprocess.run(command, check=True)
        print(f"Clip extracted to: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error extracting clip: {e}")


In [0]:
# Timestamp approach with known event extracting 10 seconds on either side of event time - in UTC.

from datetime import datetime, timedelta
import subprocess

def extract_event_clip(video_start_time_str, event_time_str, input_path, output_path, clip_duration=10):
    """
    Extracts a clip centered around an event timestamp from a video.
    Both timestamps should be in UTC format: 'YYYY-MM-DD HH:MM:SS'
    """
    # Convert strings to datetime objects
    video_start_time = datetime.strptime(video_start_time_str, "%Y-%m-%d %H:%M:%S")
    event_time = datetime.strptime(event_time_str, "%Y-%m-%d %H:%M:%S")
    
    # Calculate the offset in seconds from the video start
    time_diff = (event_time - video_start_time).total_seconds()
    
    # Center the 10s clip around the event, so go back 5s from event
    clip_start_offset = max(0, time_diff - clip_duration / 2)  # avoid negative offset

    # Run ffmpeg command
    command = [
        "ffmpeg",
        "-n",
        "-v", "quiet",
        "-stats",
        "-i", input_path,
        "-ss", str(clip_start_offset),
        "-t", str(clip_duration),
        "-q", "4",
        output_path
    ]

    try:
        subprocess.run(command, check=True)
        print(f"Clip extracted to: {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"ffmpeg error: {e}")
